In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

In [ ]:
import os
import sys
import json
from functools import reduce
from collections import OrderedDict

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("nl_core_news_sm")

In [ ]:
sources = []
for entry in os.scandir("../data/stimuleringsregeling/"):
    if entry.is_file():
        with open(entry.path) as source_file:
            source = json.load(source_file)
        if any(doc for doc in source.get("documents", []) if doc.get("text", None)):
            sources.append(source)

source_documents = [doc["text"] for source in sources for doc in source["documents"]]

kaldi_doc = """
i'm going to talk about uh <unk> which is uh a technique to have this uh more and more used in ah food microbiology especially to uh to identify incorrect <unk> [noise] um [noise] <unk> um i realize that you all have many different backgrounds so i would like to start at the very very basics of uh do you know biology so for those have already ah familiar with that it's a nice uh well i'm up at five dollars it might go a little bit far off but if you have any questions uh later on them please ask me um i'm also um available any off the news [noise] all right so do you know biology as you're probably oh no did you know that <unk> forty a genetic material that determines all of our satellite processes and that determined to get it without a factor but <unk> but the very importantly the female type [noise] and um did you know um it's usually um divided into what it's called the chromosomes in bacteria i we of course have one girl was home and he's grown was home so i made up of d._n._a. and the d._n._a. on on the d._n._a. genes are in <unk> uh which can um tend to light a two proteins [noise] um sort of d. n. a. molecule um consist of uh foreign basis um i don't mean to you i mean <unk> and <unk> and they all form a unique order for different species [noise] and uh d._n._a. it's um it's a double strength at the schools and i'm foreman he likes what a different nuclear thoughts are connected and they always form uh pairing a base pairing us at the schools <unk> shown here at at this always an a with a t energy with to see if you look into a little bit more detail at ah chemical level than what you can see our ah the teeming <unk> um via the hydrogen bombs at the schools uh every <unk> has a base ah which makes the connection with each other on and it has a sugar and a <unk> group and these are important for the connection of the different nicholas <unk> form the strength so well this um let me see so if there is a uh a bond between two um nuclear thoughts that's always a three prime and oh sweet prime ants <unk> this is one two three uh three four five [noise] and uh so so you have to five <unk> and uh that's connected at three <unk> offered to nicholas i sold i make a bombs and other schools there's <unk> it's called <unk> and that makes up the entire strangled um oh the d._n._a. alright so if you have to <unk> for pro times as i mentioned fi i protest that it's called science fiction it's it's ah copy it ah to a single strand and a single strand translated into proteins which are made up of all different um i know excess which aren't coded uh via tip lots of nuclear thoughts on that you know um i have a little ah movie just to show you the very basics or d._n._a. replication because um the player <unk> within d._n._a. replication are actually also used to <unk> you know [noise] um so now i need to get it split oh how did you get the mouse back ah [laughter] ah sorry i either because it's so it's still on the oh that's good yeah yeah okay d._n._a. it's a <unk> fifties trends twisted around each other and a devil he likes shape each jammed is made out of the <unk> the full chemical bases represented by the less as a c. g. and tea i teach john's complementary this means that whereas it as a teen one strand that will be in a in the office that stand and whoever has it as a c. that will be eighty and he had this giant eight stand has a <unk> prime and and it's free time and the two strands running opposite directions <unk> each time did the n. a. it's replicated <unk> it's a separate the two strands they sound <unk> it's done by enzyme code he may case and without seeing the formation of a replication folk separated strands each provide a template uh creating a niece trying to be n._a. and ends on cold prime <unk> stops the parents f. this end time makes a small piece of iran a cold it probably not this much the starting point to the construction at the knees trying to be n._a. and then find cold d._n._a. <unk> vines to the primary and we'll make the news john did they n._a. d._n._a. <unk> can only adds d._n._a. basis in one direction <unk> and <unk> and one of the new strains of d._n._a. <unk> it's made continuously the d._n._a. <unk> adding bases one by one in the five primed to three prime direction they understand the <unk> cannot be made in this continue this way because it runs in the opposite direction the d._n._a. <unk> connect for any make this <unk> is a small chunk <unk> <unk> each fragmented <unk> with an r. n. a. primarily d._n._a. polymer race that adds a <unk> d._n._a. bases and a five prime <unk> prime direction the next <unk> is that an added feather down the <unk> and now that i've <unk> been made and the <unk> it's repeated again once then you d._n._a. has been made the enzymes <unk> the news all the <unk> the d._n._a. and now the d._n._a. <unk> and sign then fills in the cats that i left behind his d._n._a. finally the enzymes d._n._a. like gays see about the fact meant to be in a in the stands to fall in the continue the scandals and d._n._a. replication it's describe this any conservative because h. d. n. a. molecule it's made out of one oh can fad stranded d._n._a. and one the one [noise] [noise] okay so this is the very very er basics of um do you know biology in terms of replication and now we get to the question why would you like to know the sequence of a gino um probably oh you'll have to be thinking about this to go to this lecture so why would i go and why would i uh like to know how a sequence <unk> genome secrets and how can i use it and this is something i would like to explain by um starting with the beginning of the united sequence thing because the beginning it's still something that is a very basis of all the technological advances that i now i'm going um but first of all in <unk> in general <unk> are you like to have the knowledge about it you know but also in combination with him um environmental factors as i mentioned because these are determining ah the female type unless you can see here oh hattie um we are now in this day and you see that the environment there's also playing a [noise] a role in in in our female five [noise] but um in terms of foods microbiology would like to um know did you know and it's so <unk> identifying correct rise microorganisms in food uh improving beneficial <unk> types of foods relies microsoft distaste and structure ah we would like to know the source of um pets agenda outbreaks veteran outbreaks or we would like to targets <unk> like the ones that if pathogens step past the genetic uh properties in order to um to battle them [noise] um so this is why um but now how we do that so i would like to go back to the very beginning as i mentioned nineteen seventy seven um <unk> um invented the technique uh does actually still the basis up till today and it's a very clever uh trick this is the origin of publication um if people just in nineteen seventy seven and he got the nobel prize for it to nineteen eighty because of the major break through social circle says one of the the biggest breakthroughs over the last century [noise] um i actually want to know about prize for two different subjects so very clever man um the idea is that follows so you have a piece of d._n._a. let's i._d. and i <unk> uh which is shown here you'd <unk> certificate single strength and now you have to primer assist shown in the movie that this uh needed for d._n._a. pulled him raised to make a new <unk> um she quest to copy it and this one is labeled in oh they didn't have these laws labeled with ah radioactive material [noise] and then um you make for our troops and through these troops you at the d._n._a. fragments um but also all the different nuclear thoughts and and different <unk> thoughts are usually shown was an n. n. t. p. and er you of course at the <unk> to it and you are going to at us special type of nuclear thought these are called ah terminating basis and the <unk> the auction nuclear thoughts as a shown here so this is uh at at um below you see the normal ah nuclear <unk> which is here um connected to a <unk> such that this enormous <unk> but the <unk> oh age group you'd look she talks to group to group but only an age and that means that the united <unk> cannot connect uh done an odd our nuclear thoughts to this nuclear thought so it will stop over there [noise] so these are the d. d. and d. peace and there are made for all the different uh nuclear thoughts and d._c. also adds to the same too and then the reaction is going to take place i'll show you in a in a minute the movie that that they'll show you that but the basic trick is that you've done um <unk> do you <unk> is doing his job and then you are going to <unk> uh how you use a jail [noise] <unk> <unk> and for the auto before you get four lanes and then you are going to separate the <unk> the <unk> based on their moss and our um and our um um charge and dean i of course negatively charts because of the <unk> group and the backbone so this <unk> group makes it uh negatively charge so they are going to migrate from a <unk> ah for my negative full to a <unk> full so this is uh the uh the electoral magnetic fields that you've ah see here [noise] and bought this what happens is that you're done uh because you have these d. d. off she's nipped at dies attitude to to to chew d._n._a. autographs cannot continue the sequence shot stops there so you got any answer gets all the different <unk> uh where it stopped so he <unk> this is the sequence and then um the um <unk> and then uh uh d. d. a. is building and then it stops and these smallest fragments they migrate <unk> uh the farthest into the uh along the gel so then indiana chicken simply read by going from down to up from the smallest fragments through the biggest six uh <unk> what the sequences so there's also the original trick um invented by <unk> and still uh two day uh this type of the united <unk> sequence things used that we see this is going to work d._n._a. secrets thing is the price of working at the older the building blocks oh bases innocent and if d._n._a. before we can see cleans the d._n._a. it has to be content inch of snow that pieces <unk> d._n._a. and then put into bacterial south this makes it possible to <unk> lots and lots of copies of it as a bacterial sounds multiply the d._n._a. is that isolated from the bacteria and sent fussy couldn't sing the isolated d._n._a. it's <unk> <unk> he couldn't sing reaction will take place and mixture of ingredients has added these include free d._n._a. bases a. c. g. t. d._n._a. <unk> enzyme um d._n._a. primaries modified d._n._a. bases labeled with college fluorescent tags i don't know if they landed these that cooled ten eight to bases to stop the sea couldn't sing reaction everything it's heated to ninety six degrees so fast they separates the d._n._a. entity singles john's the temperature is that <unk> fifty degrees this enables the d._n._a. prime is to <unk> to the pass needs d._n._a. the temperature is then increased to sixty degrees and the enzymes d._n._a. <unk> vines to the primary d._n._a. d._n._a. <unk> making a newsstand if d._n._a. by adding um labels d._n._a. basis to the targets d._n._a. it continues to ads d._n._a. bases until ten eight to base is added these ten minutes to basis <unk> oh to it so that no more basis can be added to the knees and if the n._a. once at ten eight to base is added the d._n._a. for them or race enzyme steps making d._n._a. and fools away from this job and everything is then he said ten ninety six degrees celsius again to separate the new d._n._a. strand from the original strand this process of heating it's cooling it's repeated again and again to produce lots of fragments that d._n._a. of different like the lengths that each fragment depends on when it tena database <unk> added to read the secrets of the d._n._a. the various fragments of separated by names using a price at school and that <unk> <unk> <unk> she went at the tight end and then that took her challenges that site they school days the negativity chives d._n._a. motor skills to me through the <unk> opinion each <unk> each tip perry contains at porous jail <unk> d._n._a. move to the gym more easily than the longer d._n._a. fragments as a result the fragments become arranged by size and the shortest to the longest i've the d._n._a. <unk> come to the end of the <unk> and they used that makes the ten minutes it basis like up to canada is detected by a camera and recorded each ten <unk> with a different kind of a <unk> green see believe gee yeah i've i'm tea rides the show taste d._n._a. fragments will be red fast and the youngest red lost the sea couldn't sing machine records the kind of at the time and they said bases as a series that's kind of that's been the hawks each <unk> represents that labeled ten minutes a base at the end of each fragment of d._n._a. <unk> the kind of intellect says we get the sequence although piece of d._n._a. okay so i i think you have noticed that <unk> the technique that is um in a side of than this movie is a little bit different than what i just uh told you about about four different lanes and that's because this is <unk> uh technique wedded don't use the um the radioactive label off the primer to distinguish between the uh the fragments uh in a different lanes but they use lives so a lot so few lessons label and this is something that has used uh <unk> uh today um so there's already a showing you'd have this these kind of technologies are uh all the time on the development [noise] but at this all based on this invention by uh by a fairly like soccer then in the nineteen nineties so that's ah continue with the timeline um a technique was introduced which school to shut the gun sequence sing and uh the basic idea is to use a certain enzymes to cut the d._n._a. and too many different fragments all the different fragments and then indeed clone these old into fact doors well <unk> uh small pieces of d._n._a. sheesh ashville shown in the movie in the beginning and then these were added to bacteria amplify and that has <unk> again um extract that uh <unk> older fragments <unk> and now you've got a lot of different fragments as a shown here and around that time also buy from ethics as strong computer power walls are getting um getting developed a in a way that you can have so many different fragments sequence that they start to over <unk> and because of this over laying uh nature of all the different sequence uh fragments you can actually assemble er the gino so by from ethics uh <unk> uh especially in the beginning very much used to assemble older sequence fragments to get her into a overall um uh gino [noise] of course if the genome off a species just completely uh known it's very difficult to assemble all these different uh fragments so that's of course uh took a long time or that takes more time to uh sequence of gino which is completely a known if you for example <unk> sequence <unk> or a salmonella uh you can use it as a template too um let's say a line all the <unk> like you have found for i knew a <unk> so much easier than when you have to <unk> do you know more of a species says unknown [noise] and the other schools also called a template generation so sometimes you force bases you already have a template uh sometimes you don't have it so the first uh <unk> uh <unk> honest nineteen ninety five is the first complete ah gino over bacterium uh influenza uh then we got the nineteen ninety eight uh the warm genome sequence in two thousand to food fly and also human so that was uh only like um seventeen years ago when we got the first human genome assembled then around that time also got a new stage of the united <unk> which is called the next generation a sequence sing or also called <unk> <unk> the idea is they are too upscale this uh song i technique for example in a way that you can <unk> many <unk> at the same time so parallel to each other [noise] that is the major aim of this uh let's say a dime points in in in the development of <unk> different companies uh work on this like pacific buy sciences <unk> different uh machines as you can see here um so as i mentioned the major aim of these techniques is to uh <unk> most of the sequence many assembles at the same time and <unk> as well so it becomes more complex over time and i want to show you one of these uh <unk> movie and other <unk> the <unk> company looming ah [noise] it's a bit more complicated than to once before so um i will try to um uh some stages i will i will stop the movie and explain if need it so that's how to look [noise] something worked for a while it's component of forum basic stat sample <unk> [noise] class or generation [noise] england thing [noise] and data analysis [noise] there are a number of different ways to prepare sample all preparation methods add a doctor's [noise] to the end that would be in a fragment [noise] river do cycling suffocation edition teeth are introduced to think of anything dining site indices and regions complimentary to the <unk> so these are adults that are uh associate it through the sequence that you want to uh sequence so the green wall and this for the primer uh the <unk> is an index ah shot that's also sequence another storm too to distinguish between all the different samples if you are going to do because you cannot paralyze uh the different sampling and that's the idea so you need to you need to um a lot so you need to connect older fragments duties adapters and these uh blue uh nuclear thought sequence is too um aligned to to to chip that that you will see in a minute [noise] last terrain it's a process where each fragment <unk> well it's certainly amplify the first out is it the last slide living lane [noise] each lane is a channel coated with online component of two types of all like a [noise] hyper division is enable by the first of the two types of august i'm a survey [noise] this on ago it's complimentary the adaptor region on one of the fragments france [noise] upper limit race creed to complement as a hybrid ice fragment [noise] been demonstrated molecule majored in the original template is washed away [noise] this <unk> i don't really amplified through break application [noise] in this process the strength phones over in the adaptor region hybrid items to the second type of long ago i'm a close out [noise] <unk> generate the complimentary strand forming a double stranded branch [noise] this bridges teenagers resulting into singles traded copies of the molecule that are together now [noise] the process is then repeated over and over and the curse simultaneously for a million <unk> of all the fragment so hey <unk> there's a difference between the techniques before and now in the in the beginning all these fragments were clone into little uh d._n._a. sequences transformed into bacteria and <unk> but now this is all done on the chip so that is a major improvement because uh uh <unk> uh <unk> uh all these steps in between <unk> necessary anymore you can amplify loft and that's very important important um development [noise] after break application the reverse strands are clean and washed up leaving only to forward strange [noise] the three prime ends or blocked to prevent unwanted climbing [noise] figuring thing began with the extension of the first big one thing primary to produce the first three [noise] with each cycle fluorescent like pet nuclear type compete for addition to the growing chain [noise] only one is incorporated based on the sequence at the time [noise] after the edition of each nuclear tied the customers are excited by enlightened oregon and the characteristics <unk> isn't it [noise] this proprietary process it's called sequence <unk> license doesn't [noise] the number of cycles determines the length of three [noise] the emission wave length along with the signal intensity determines the base call [noise] for a given class or oh identical strand i read simultaneously [noise] hundreds of millions of clusters or <unk> in a massively parallel process [noise] image represent the small fraction at the first out [noise] after the completion of the first read read product gift washed away [noise] and this that the index one read primaries entertaining and hybrid eyes to the temperature [noise] to read is generated similar to the first three [noise] after completion of the index read read product is washed off and the three prime ends of the template r. d. protected [noise] the template now full dover and binds the second all ago on this list that index too it's right in the same manner is indexed one [noise] lemme raises extend the second slows how long ago forming a double stranded bridge this double standard d._n._a. and then letting your eyes and the three prime ends or blah [noise] the original forward strand cleaned up and washed away leaving only the reverse strange [noise] read too begins introduction at the <unk> tennessee clinton primer athletes read one instead of to repeat it until the desire to read like there's a team [noise] the redo product it's been washed away [noise] this entire process generate millions every representing all the fragment [noise] he clinton's from a little sample libraries i separated based on that unique indices introduce during the sampled preparation for each sample weights but similar stretches of these calls i locally cluster [noise] forward and reverse read our parents creating continuous sequences [noise] these contiguous sequences arline adopted a reference she now for their yet identification [noise] their parents and information is used as a result ambiguous on line [noise] okay so this might be a lot of information and it's quite complicated so off the words you can have uh an all the <unk> by the by the the execs sequences of events but that that whole mess of tastes here that these techniques are developed to ah well two <unk> two sequence many samples at the same time to sequence both events seem <unk> off of each other and you can also imagine that the more i reach you sequence so it's so it's always a <unk> sort of more <unk> more times you <unk> uh um uh i read the more the higher quality of the sequence isn't that also relates to the cost of the of the entire procedure [noise] so this technique did not necessarily a result in vets are um sequence saying but of course <unk> and and more so uh hi to protest it this cold [noise] and also if they did use to <unk> uh enormously as you can see er um [noise] at the left uh the costs are genome death <unk> around two thousand four hundred million euros for one do you know and uh around the time on this <unk> introduced this dropped dramatically and and now in two thousand fifteen we are about ah <unk> the sequence as you know so that is ah quite um an improvement and at the at the right you can see the different sizes of the <unk> all different species so costs there's also an important um facts are here um uh which uh which is linked to the major development that i would like to introduce the final uh she couldn't sing technique this is the latest developments and hold you know sequence thing and it is called a <unk> that was actually into <unk> actually started to interviews already in the nineties in nineteen ninety five they started with this <unk> er <unk> last year there was a big break through in this also i'd lost um named as one of the biggest uh breakthroughs of two thousand sixteen by the general science and it's russia's follows you have um a membrane and then there's <unk> particular protein uh um <unk> which is a poor and food is for one single d._n._a. can go and here at this um this position at um at the <unk> the rat line um <unk> um <unk> um uh capability of the ends on to uh measuring a certain currents change and every molecule is a little bit different at the chemical level and they also have a different occurrence ah also to different uh basis of the <unk> so by this little uh difference uh end occurrence across this uh <unk> you can determine um the nuclear thought sequence largest measuring the currents change across this membrane across this protein which is showing here and this has now been developed into i've very small uh device this is a huge be um um connect i saw it shows a little bit how small the defies s. and all the chip you can uh put your sample and sequence um you know do you know you can connect the dark until your computer [noise] um and the advantages art or that there's a single d. n. a. molecule so you can also because the chip has many pores you can actually see grants at the same time any different to your name on accuse so for example from a sample of many different microorganisms you can uh civil thing there's they figure out what types of microorganisms are in my simple you can do this real time and uh it's a mobile the fly so you can take it with you and you can sit in the bush and secrets your samples over there and actually the reason why it's got uh this breakthrough off two years because i bought a uh issue um and and there are many laps used this technique um in africa the sequence ah i bought a [noise] and to correct rise to strings so this is the latest developments [noise] now we know a little bit more about different uh <unk> uh <unk> uh techniques there's this one application of the <unk> and an out a hospital outbreak of salmonella and within only one day they knew what type of <unk> well then at the <unk> but also what type of salmonella loss and that was also used to figure out uh one of the sources outbreak walls and also to um to directly gets protocols all how to treat the ah the patients that are in dallas and our hospitals [noise] so do you um the idea is that <unk> because you're kind of very rapidly i determine <unk> actually do not have the sequence inside you know because you can stop at some points because the computer can tell you already at some point what type of micro organism it is based um searching into a database alright so do you know she couldn't sing as many different applications of course i already <unk> in terms of infectious diseases food safety which is a key here uh personalized medicine to have a specific uh medicine for people but different the d._n._a. of backgrounds lands breeding of course optimize that <unk> engineering so many different applications [noise] um but i would like to focus um for the last five minutes five ten minutes on food safety and how you can use these techniques to identify ah pathogens and focus on this <unk> this daily uh uh let's say initiative to construct the idea would be in the future to construct databases ah for different um pathogens um enrich your store <unk> genome sequence plus all all kinds of data associate it to <unk> like the clinical data or source of infection or monitor the female dives and these databases to got to buy 'em from ethics approaches um condemn be used to quickly find out what type of that's a gym is in my food for example and uh <unk> uh to be off a use for finding ah the source of these outbreaks more rapidly and while just on uh nowadays um just this is just um just slide to show you that that people work on uh they <unk> they determine hold you know sequences off <unk> ah that for example in disguise came from um i'm an outbreak and kind of uh two thousand eight and a death reveals that there were a genetic differences between these isolates you could only find these by sequence in the entire gino with other techniques um and then based on the gentleman differences to investigate it fit our ah into clinical an environmental have food <unk> and that's just hold it in the conclusion that actually three types of passage ins were involved in this outbreak and this type of information is then all start in in these databases and the idea would then be two um in in the future if you have isolates you determine that uh the d._n._a. sequence of it if you search entered the database if there are already uh strange known that are very similar or are the same such as you can correct arise these strange and that you can it can help you to locate the source of the infection [noise] um this idea has been implemented especially in the united states now since two thousand threatened if we talk about syria they start all ah from human disease guy says um they're the gino <unk> and associated that into these into these databases [noise] and this initiative uh according to ah to develop farce results in his shown here um in this figure so at the y. access you see the number of cases um that that's that's overtime because of the whole genome secrets sing approaching the database construction <unk> more outbreaks and i also found fewer cases ah in outbreak because they found out the source of the the outbreak more quicker than without the approach ah using the database construction so that's an important development and by the gym correct the recession and and ah source identification this is an um an example of that from a routine um test uh just uh ice lights were taken from my hispanic style cheese produced by uh <unk> uh a license brent [noise] andy <unk> genome sequences that were um determines uh from these <unk> period into this uh database and then i found a heads with five <unk> that has very similar um this area indicted disease frito type and actually ah two of these people died and they will also it's also known that day actually um have eaten a article <unk> this this this specific ah type of cheese also from the same brands so and you end up conclusion game <unk> not to eat this um this type of cheese and all over over the different <unk> morals these cases are are are are known i'm no choice you can read it but for example in two thousand sixteen there were a number of cases in which holds you know sequence thing um and in combination with this database or help to find uh outbreak uh <unk> more quicker and this is not only um <unk> <unk> this is especially in the united states and in um in europe this will also be um initialized uh very soon [noise] so i will leave it by this so uh so there are some whole messages so d._n._a. sequence ink on reveal the genetic signatures underlying <unk> ah the first <unk> methods mostly followed by <unk> and it was also the [noise] the the basis for all the the other techniques that were developed afterwards well you can use it to characterize but the genetic strains and that can also or many calls have you to finding the source and also to find the source quicker [noise] um but still you need to also have all the types of information to really pinpoint where to outbreak walls and hold g._m._c. fencing can be used for that
"""
source_documents.append(doc)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english")
tfidf_vectorizer.fit(source_documents)
feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
noun_documents = []
nouns = [token.text for token in nlp(doc) if token.pos == spacy.parts_of_speech.NOUN] 
noun_documents.append(" ".join(nouns))

vectors = tfidf_vectorizer.transform(noun_documents)
source_vector = vectors.sum(axis=0).A1
highest_tfidf_indexes = source_vector.argsort()[-10:]
keywords = OrderedDict(
    (feature_names[ix], source_vector[ix],)
    for ix in highest_tfidf_indexes
)

print(list(reversed(keywords)))